###Importing libraries

In [ ]:
import numpy as np
import pandas as pd

###Importing dataset (from google drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('./drive/MyDrive/Project1_Sentiment_Analysis/ForAkash_new_reviews.csv', delimiter = ',', quoting = 0, encoding='cp1252')
dataset.head()

,Review
0,2 goldfish-2 gave goldfish-1 7$ because sadly ...
1,4 goldfish-1 did not attend many labs. even th...
2,3 goldfish-4 and goldfish-3 contributed pretty...
3,1 no comment
4,2 no comment


###Data cleaning

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus=[]

for i in range(0, 135):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [ ]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='./drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [ ]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(135, 1420)

###Predictions (via sentiment classifier)

In [ ]:
import joblib
classifier = joblib.load('./drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c2_Classifier_Sentiment_Model')

In [ ]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[-1  1  1  0  0  1  1  1  1  0  1  1  0  0  1  1  1  1  0  0  1  1  0  1
  1  1  1  0  1  0  1  1  1  1  1  1  1  1  1  1  0  1  0  1  0  1  0  1
 -1  1  1  1  1  0  1  1  1  1  1  1  0  0  1  0  0  0  1  1  1  1  1  0
  0  0  1  0  0  0  1  1  1  1  0  0  0  0  0  0  0  0  0  1  1  1  1  0
  1  1  1  1  1  1  1  1  0  1  0  1  1  0  0  1  1  1  1  1  1  1  1  0
  0  0  0  0  0  1  1  1  0  0  1  1  0  0  1]


In [ ]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head()

,Review,predicted_label
0,2 goldfish-2 gave goldfish-1 7$ because sadly ...,-1
1,4 goldfish-1 did not attend many labs. even th...,1
2,3 goldfish-4 and goldfish-3 contributed pretty...,1
3,1 no comment,0
4,2 no comment,0


In [ ]:
dataset.to_csv("./drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c3_Predicted_Sentiments_Fresh_Dump.tsv", sep='\t', encoding='UTF-8', index=False)